In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, LSTM
import os

# LSTM (dataset non complété)

In [2]:
data = pd.read_csv('finaldataset.csv', index_col = 0, header=0, encoding='utf-16', dtype=int)
data.head()

,1ere,relecture,gt,consommation,yohan,temps,dire,clair,mettre,decibel,...,rcsup,noyer,redetailler,brosset,faculte,lavoisier,lepaumier,3xx5521b,minuscule,demande_de_support
0,1,1,1,1,1,1,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
unique_words = data.drop('demande_de_support', axis = 1).columns.to_list()
word_to_idx = {
    unique_word:idx for idx,unique_word in enumerate(unique_words)
}

True

In [4]:
# fix random seed for reproducibility
np.random.seed(777)

In [30]:
voc_size = len(unique_words)
embedding_vector_features=45

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(128,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 45)          466155    
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 128)         89088     
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)               

In [44]:
X = data.loc[data["demande_de_support"] != 2].drop(columns=['demande_de_support'])
y = pd.Series(data.loc[data["demande_de_support"] != 2]['demande_de_support'], dtype=int)

### Entrainement avec les vecteurs sparse

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [46]:
model.fit(X_train,y_train,validation_data=(X_val, y_val), epochs=1, batch_size=128)

In [32]:
model.evaluate(X_test,y_test)

20/20 [==============================] - 6s 319ms/step - loss: 0.3905 - accuracy: 0.8642


[0.3905010223388672, 0.8642172813415527]

### Entrainement avec les séquences

In [16]:
X_seq = [ [k for k in range(len(mail)) if mail[k]>0] for mail in X.values ]
MAX_LENGTH = max([ len(seq) for seq in X_seq])
X_seq = pad_sequences(X_seq, maxlen = MAX_LENGTH)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [10]:
model.fit(X_train,y_train,validation_data=(X_val, y_val), epochs=1, batch_size=128)

14/14 [==============================] - 99s 7s/step - loss: nan - accuracy: 0.8051 - val_loss: nan - val_accuracy: 0.8562


In [11]:
model.evaluate(X_test,y_test)

20/20 [==============================] - 6s 320ms/step - loss: nan - accuracy: 0.8642


[nan, 0.8642172813415527]

In [12]:
input_mail = """
    DRS - 
    Bonjour
    j'ai un souci avec le décisionnel SYNERGIE, il y a un problème de référenciel.

    Cordialement,
    Martin
    """

In [42]:
lem_mail = ['drs', 'bonjour', 'souci', 'decisionnel', 'decibel', 'probleme', 'ref']
seq = [[word_to_idx[word] for word in lem_mail]]
input_seq = pad_sequences(seq)
input_seq

array([[3934,   50,  237,   18,    9,    9,  222, 2181]])

In [43]:
model.predict(input_seq)

array([[0.6208053, 0.3791947]], dtype=float32)